---

# Chapter 14: UI and Separation Patterns (MVC, MVVM, MVP)

## Opening Context

User interfaces are among the most volatile parts of a software system. Requirements change, new devices emerge, and design trends evolve. If UI logic is tightly coupled with business logic, every change risks breaking core functionality, and automated testing becomes nearly impossible. The need to separate the **presentation layer** from the **domain layer** has driven the creation of several architectural patterns specifically for user interfaces.

This chapter explores three classic patterns that organise the interaction between the user, the UI, and the underlying data:

1. **Model‑View‑Controller (MVC)** – the original pattern from Smalltalk that separates input, presentation, and domain logic.
2. **Model‑View‑ViewModel (MVVM)** – an evolution that leverages data binding to simplify UI development, popularised by WPF and now used in many modern frameworks.
3. **Model‑View‑Presenter (MVP)** – a pattern that makes the view completely passive, maximising testability.

Each pattern addresses the same fundamental problem but makes different trade‑offs in terms of coupling, testability, and platform suitability. Understanding them equips you to choose the right approach for your UI framework and to communicate design decisions with your team.

---

## 14.1 Model-View-Controller (MVC)

### Intent
*Separate an application into three interconnected components: the **Model** (data and business rules), the **View** (user interface), and the **Controller** (input handling and flow control). The goal is to allow multiple views of the same data and to decouple changes in one component from affecting others.*

### The Problem

Before MVC, UI code often mixed everything together: event handlers directly updated the database, and the screen layout was interwoven with data retrieval. This made applications:

- **Difficult to maintain** – a change in the UI required changes in the data logic and vice versa.
- **Hard to test** – you couldn't test business rules without rendering the UI.
- **Inflexible** – supporting a second type of view (e.g., both desktop and mobile) meant duplicating code.

### The Solution: MVC

MVC divides the application into three parts:

- **Model** – Manages the data, business rules, and logic. It is independent of the UI. It notifies observers (usually Views) of changes.
- **View** – Renders the Model’s data. It queries the Model and updates its presentation when the Model changes.
- **Controller** – Handles user input (clicks, keystrokes). It interprets input, updates the Model, and sometimes selects a View.

In the classic Smalltalk MVC, the View and Controller are tightly coupled (one Controller per View), but modern interpretations vary. The key is the separation of concerns and the **Observer** relationship between Model and View.

### Example: A Simple Todo List (Console/Web)

We'll illustrate MVC with a simple todo list application using TypeScript. The Model manages an array of todos and notifies observers of changes. The View displays the todos. The Controller handles user commands.

#### Model (with Observer Pattern)

```typescript
// model/todo.model.ts
type Observer = () => void;

export class TodoModel {
  private todos: string[] = [];
  private observers: Observer[] = [];

  addTodo(task: string): void {
    this.todos.push(task);
    this.notifyObservers();
  }

  removeTodo(index: number): void {
    this.todos.splice(index, 1);
    this.notifyObservers();
  }

  getTodos(): string[] {
    return [...this.todos]; // return a copy to prevent external mutation
  }

  // Observer pattern methods
  addObserver(observer: Observer): void {
    this.observers.push(observer);
  }

  private notifyObservers(): void {
    this.observers.forEach(obs => obs());
  }
}
```

**Explanation**:
- The Model holds the data (`todos`) and business logic (add/remove). It is completely unaware of the View or Controller.
- It implements the **Observer** pattern so that Views can register for updates. When the data changes, all observers are notified.

#### View

```typescript
// view/todo.view.ts
import { TodoModel } from '../model/todo.model';

export class TodoView {
  constructor(private model: TodoModel) {
    // Register with the model to be notified of changes
    this.model.addObserver(() => this.render());
  }

  // Render the current todos (e.g., to console or DOM)
  render(): void {
    console.clear();
    console.log('=== Todo List ===');
    this.model.getTodos().forEach((todo, index) => {
      console.log(`${index + 1}. ${todo}`);
    });
    console.log('\nCommands: add <task> | remove <index> | exit');
  }

  // Display a message (e.g., error)
  showMessage(msg: string): void {
    console.log(msg);
  }
}
```

**Explanation**:
- The View holds a reference to the Model (though in some MVC variants it only knows the Controller).
- It registers itself as an observer of the Model. Whenever the Model changes, `render()` is called automatically.
- The View is responsible for displaying data and providing a way for the user to input commands (here via console).

#### Controller

```typescript
// controller/todo.controller.ts
import { TodoModel } from '../model/todo.model';
import { TodoView } from '../view/todo.view';
import * as readline from 'readline';

export class TodoController {
  private rl: readline.Interface;

  constructor(private model: TodoModel, private view: TodoView) {
    this.rl = readline.createInterface({
      input: process.stdin,
      output: process.stdout
    });
  }

  // Start the application loop
  start(): void {
    this.view.render();
    this.promptUser();
  }

  private promptUser(): void {
    this.rl.question('> ', (input) => {
      const [command, ...args] = input.trim().split(' ');
      const arg = args.join(' ');

      switch (command) {
        case 'add':
          if (arg) {
            this.model.addTodo(arg);
          } else {
            this.view.showMessage('Please provide a task.');
          }
          break;
        case 'remove':
          const index = parseInt(arg) - 1;
          if (!isNaN(index) && index >= 0 && index < this.model.getTodos().length) {
            this.model.removeTodo(index);
          } else {
            this.view.showMessage('Invalid index.');
          }
          break;
        case 'exit':
          this.rl.close();
          return;
        default:
          this.view.showMessage('Unknown command.');
      }
      this.promptUser();
    });
  }
}
```

**Explanation**:
- The Controller handles user input. It interprets the command and updates the Model accordingly.
- It also uses the View to display messages (e.g., errors). In some MVC interpretations, the Controller may update the View directly, but here we keep the View observing the Model for automatic updates.
- The Controller does **not** contain business logic; it delegates to the Model.

#### Composition (Main)

```typescript
// main.ts
import { TodoModel } from './model/todo.model';
import { TodoView } from './view/todo.view';
import { TodoController } from './controller/todo.controller';

const model = new TodoModel();
const view = new TodoView(model);
const controller = new TodoController(model, view);

controller.start();
```

**Explanation**:
- The main function wires the components together. The Model is independent; the View observes the Model; the Controller orchestrates input and updates the Model.
- This separation allows us to change the View (e.g., to a web interface) without altering the Model or Controller logic.

### Benefits of MVC

- **Separation of Concerns** – Each component has a distinct responsibility.
- **Multiple Views** – The same Model can have several Views (e.g., list view and detail view) because the Model notifies all observers.
- **Maintainability** – Changes to the UI typically only affect the View and Controller, not the Model.
- **Testability** – The Model can be unit‑tested independently; the Controller can be tested with mock Model and View.

### Drawbacks

- **Complexity** – For simple UIs, MVC can introduce unnecessary indirection.
- **Controller‑View Coupling** – In many implementations, the Controller is tightly coupled to a specific View, making it harder to reuse.
- **Fat Controllers** – Often, too much logic ends up in the Controller, making it hard to test and maintain.

MVC has many variations (e.g., Model‑View‑Adapter, Model‑View‑Presenter). The next patterns address some of its shortcomings.

---

## 14.2 Model-View-ViewModel (MVVM)

### Intent
*Separate the development of the graphical user interface from the business logic by introducing a **ViewModel** that acts as a converter between the Model and the View, leveraging data binding to keep them synchronised automatically.*

### The Problem

In MVC, the Controller often contains code to update the View based on Model changes. This works, but it can become repetitive, especially when the View has many interactive elements. Developers wanted a way to **automate** the synchronization between the View and the Model, reducing boilerplate. Additionally, testing the Controller still required simulating user input.

### The Solution: MVVM

MVVM, popularised by Microsoft for WPF and later adopted in frameworks like Vue.js, Knockout, and Angular, introduces a **ViewModel**:

- **Model** – Same as in MVC: the domain data and business logic.
- **View** – The visual elements (UI). It is **passive** and unaware of the Model.
- **ViewModel** – An abstraction of the View that exposes data and commands suitable for binding. It holds the state of the View and handles presentation logic. It communicates with the Model.

The key innovation is **data binding**: the View and ViewModel are automatically synchronised via a binding framework. When a property in the ViewModel changes, the View updates automatically; when the user interacts with the View, the corresponding command in the ViewModel is invoked.

### Example: Todo List with MVVM (Vue-like)

We'll simulate a simple MVVM implementation using a hypothetical binding library to illustrate the concepts.

#### Model

```typescript
// model/todo.model.ts (same as before)
export class TodoModel {
  private todos: string[] = [];
  private observers: (() => void)[] = [];

  addTodo(task: string): void { this.todos.push(task); this.notify(); }
  removeTodo(index: number): void { this.todos.splice(index, 1); this.notify(); }
  getTodos(): string[] { return [...this.todos]; }
  addObserver(obs: () => void): void { this.observers.push(obs); }
  private notify(): void { this.observers.forEach(o => o()); }
}
```

#### ViewModel

```typescript
// viewmodel/todo.viewmodel.ts
import { TodoModel } from '../model/todo.model';

export class TodoViewModel {
  // Observable properties (simplified with getters/setters and change notification)
  private _newTask: string = '';
  private _todos: string[] = [];
  private _errorMessage: string = '';

  // We'll simulate data binding by having a change listener
  private changeListeners: (() => void)[] = [];

  constructor(private model: TodoModel) {
    // Observe model changes and update our own _todos
    model.addObserver(() => {
      this._todos = model.getTodos();
      this.notifyChange();
    });
    // Initial load
    this._todos = model.getTodos();
  }

  // Bindable properties
  get newTask(): string { return this._newTask; }
  set newTask(value: string) {
    this._newTask = value;
    this.notifyChange();
  }

  get todos(): string[] { return this._todos; }
  get errorMessage(): string { return this._errorMessage; }

  // Commands (methods that the View can bind to)
  addTask(): void {
    if (this._newTask.trim() === '') {
      this._errorMessage = 'Task cannot be empty.';
      this.notifyChange();
      return;
    }
    this.model.addTodo(this._newTask);
    this._newTask = ''; // clear input
    this._errorMessage = '';
    this.notifyChange(); // because newTask and errorMessage changed
  }

  removeTask(index: number): void {
    if (index >= 0 && index < this._todos.length) {
      this.model.removeTodo(index);
    } else {
      this._errorMessage = 'Invalid index.';
      this.notifyChange();
    }
  }

  // Binding support (simplified)
  bind(callback: () => void): void {
    this.changeListeners.push(callback);
  }

  private notifyChange(): void {
    this.changeListeners.forEach(cb => cb());
  }
}
```

**Explanation**:
- The ViewModel holds **observable properties** (`newTask`, `todos`, `errorMessage`). When these change, it notifies listeners (the View).
- It contains **commands** (`addTask`, `removeTask`) that encapsulate user actions. These commands update the Model and then adjust the ViewModel’s state.
- The ViewModel observes the Model and keeps its `todos` property in sync. This decouples the View from the Model.

#### View (with Hypothetical Binding)

```html
<!-- view/todo.view.html (simplified) -->
<div id="app">
  <h1>Todo List</h1>
  <input type="text" data-bind="value: newTask" placeholder="New task">
  <button data-bind="click: addTask">Add</button>
  <div data-bind="text: errorMessage" style="color: red;"></div>
  <ul>
    <li data-bind="foreach: todos">
      <span data-bind="text: $data"></span>
      <button data-bind="click: $parent.removeTask.bind($parent, $index)">Remove</button>
    </li>
  </ul>
</div>
```

```typescript
// view/todo.view.ts (binding setup)
import { TodoViewModel } from '../viewmodel/todo.viewmodel';

export class TodoView {
  private viewModel: TodoViewModel;

  constructor(viewModel: TodoViewModel) {
    this.viewModel = viewModel;
    this.initBindings();
  }

  private initBindings(): void {
    // In a real framework like Vue, this is done declaratively.
    // Here we simulate by manually updating DOM when ViewModel changes.
    const app = document.getElementById('app')!;
    const input = app.querySelector('input') as HTMLInputElement;
    const addButton = app.querySelector('button') as HTMLButtonElement;
    const errorDiv = app.querySelector('div[data-bind="text: errorMessage"]') as HTMLDivElement;
    const ul = app.querySelector('ul')!;

    // Bind input value to viewModel.newTask
    input.addEventListener('input', (e) => {
      this.viewModel.newTask = (e.target as HTMLInputElement).value;
    });

    // Bind button click to viewModel.addTask
    addButton.addEventListener('click', () => {
      this.viewModel.addTask();
    });

    // Update UI when ViewModel changes
    this.viewModel.bind(() => {
      // Update input value (if it changed programmatically)
      input.value = this.viewModel.newTask;
      // Update error message
      errorDiv.textContent = this.viewModel.errorMessage;
      // Re-render todo list (simplified: rebuild entire list)
      ul.innerHTML = '';
      this.viewModel.todos.forEach((todo, index) => {
        const li = document.createElement('li');
        li.innerHTML = `<span>${todo}</span> <button>Remove</button>`;
        li.querySelector('button')!.addEventListener('click', () => {
          this.viewModel.removeTask(index);
        });
        ul.appendChild(li);
      });
    });

    // Initial render
    this.viewModel.bind(() => {})(); // trigger first update
  }
}
```

**Explanation**:
- The View sets up bindings between DOM elements and the ViewModel. In a real framework, this would be declarative and automatic.
- When the user types in the input, the View updates the ViewModel's `newTask`. When the ViewModel notifies change, the View updates the DOM (input value, error message, list).
- The View contains **no business logic**; it only reacts to ViewModel changes and forwards user actions to the ViewModel.

#### Composition

```typescript
// main.ts
import { TodoModel } from './model/todo.model';
import { TodoViewModel } from './viewmodel/todo.viewmodel';
import { TodoView } from './view/todo.view';

const model = new TodoModel();
const viewModel = new TodoViewModel(model);
const view = new TodoView(viewModel);
// View is now active and bound
```

### Benefits of MVVM

- **Declarative UI** – Data binding reduces boilerplate code for synchronising View and ViewModel.
- **Testability** – The ViewModel can be unit‑tested without a UI (just plain objects and functions).
- **Separation** – The View is completely passive; designers can work on the UI while developers work on the ViewModel.
- **Platform Adaptability** – The same ViewModel can be used with different Views (e.g., web, mobile) as long as the target platform supports binding.

### Drawbacks

- **Overhead** – Data binding frameworks can be complex and may introduce performance issues if not used carefully.
- **ViewModel Bloat** – The ViewModel can become large if it handles too much presentation logic.
- **Learning Curve** – Understanding data binding and observable patterns takes time.

MVVM is especially powerful in platforms with rich data binding support, such as WPF, Xamarin, Vue.js, and Knockout.

---

## 14.3 Model-View-Presenter (MVP)

### Intent
*Separate the presentation logic from the View by introducing a **Presenter** that acts as the middleman. The View is completely passive and delegates all user interactions to the Presenter, which updates the Model and then refreshes the View.*

### The Problem

In MVC, the View often observes the Model directly. This can lead to Views that are still somewhat coupled to the Model (e.g., they know about Model properties). For maximum testability, we want the View to be a **passive** interface that can be easily mocked. The Presenter should contain all presentation logic and be fully testable without any UI framework.

### The Solution: MVP

MVP defines three components:

- **Model** – The domain data and business logic (same as before).
- **View** – A passive interface that exposes methods for displaying data and raising user events. It knows nothing about the Model.
- **Presenter** – Retrieves data from the Model, formats it for display, and updates the View. It also handles events from the View, updates the Model, and refreshes the View.

MVP has two main variants:

- **Passive View** – The View is as dumb as possible; it only displays data and forwards events. The Presenter does all the work.
- **Supervising Controller** – The View may handle simple data binding, but complex logic remains in the Presenter.

We'll focus on **Passive View** for maximum testability.

### Example: Todo List with MVP

#### Model (unchanged)

```typescript
// model/todo.model.ts
export class TodoModel {
  private todos: string[] = [];
  addTodo(task: string): void { this.todos.push(task); }
  removeTodo(index: number): void { this.todos.splice(index, 1); }
  getTodos(): string[] { return [...this.todos]; }
}
```

#### View Interface (Abstraction)

```typescript
// view/todo.view.interface.ts
export interface TodoView {
  setTodos(todos: string[]): void;
  setErrorMessage(message: string): void;
  clearInput(): void;
  // Events (usually implemented as callbacks or observables)
  onAddTask: (task: string) => void;
  onRemoveTask: (index: number) => void;
}
```

**Explanation**:
- The View interface defines **what the View can do** (display data) and **what events it raises** (user actions).
- The Presenter will depend on this interface, not a concrete View, allowing it to be tested with a mock.

#### Concrete View (Console)

```typescript
// view/console-todo.view.ts
import { TodoView } from './todo.view.interface';
import * as readline from 'readline';

export class ConsoleTodoView implements TodoView {
  onAddTask: (task: string) => void = () => {};
  onRemoveTask: (index: number) => void = () => {};

  private rl: readline.Interface;

  constructor() {
    this.rl = readline.createInterface({
      input: process.stdin,
      output: process.stdout
    });
    this.promptUser();
  }

  setTodos(todos: string[]): void {
    console.clear();
    console.log('=== Todo List ===');
    todos.forEach((todo, i) => console.log(`${i+1}. ${todo}`));
    console.log('\nCommands: add <task> | remove <index> | exit');
  }

  setErrorMessage(message: string): void {
    console.log(message);
  }

  clearInput(): void {
    // Not needed for console, but we could simulate.
  }

  private promptUser(): void {
    this.rl.question('> ', (input) => {
      const [cmd, ...args] = input.trim().split(' ');
      const arg = args.join(' ');
      if (cmd === 'add') {
        this.onAddTask(arg);
      } else if (cmd === 'remove') {
        const idx = parseInt(arg) - 1;
        if (!isNaN(idx)) this.onRemoveTask(idx);
        else this.setErrorMessage('Invalid index');
      } else if (cmd === 'exit') {
        this.rl.close();
        return;
      } else {
        this.setErrorMessage('Unknown command');
      }
      this.promptUser();
    });
  }
}
```

**Explanation**:
- The concrete View implements the interface. It calls the presenter via `onAddTask` and `onRemoveTask` callbacks.
- The View does **no** business logic; it just displays what the Presenter tells it to and forwards user input.

#### Presenter

```typescript
// presenter/todo.presenter.ts
import { TodoModel } from '../model/todo.model';
import { TodoView } from '../view/todo.view.interface';

export class TodoPresenter {
  constructor(private model: TodoModel, private view: TodoView) {
    // Connect view events to presenter methods
    this.view.onAddTask = (task) => this.addTask(task);
    this.view.onRemoveTask = (index) => this.removeTask(index);
    this.refreshView();
  }

  private addTask(task: string): void {
    if (task.trim() === '') {
      this.view.setErrorMessage('Task cannot be empty.');
      return;
    }
    this.model.addTodo(task);
    this.view.clearInput();
    this.refreshView();
  }

  private removeTask(index: number): void {
    if (index >= 0 && index < this.model.getTodos().length) {
      this.model.removeTodo(index);
      this.refreshView();
    } else {
      this.view.setErrorMessage('Invalid index.');
    }
  }

  private refreshView(): void {
    this.view.setTodos(this.model.getTodos());
  }
}
```

**Explanation**:
- The Presenter holds references to both Model and View (via interfaces).
- It subscribes to View events and handles them, updating the Model and then calling `refreshView()` to update the View.
- It contains **all presentation logic** (validation, formatting, navigation).
- The Presenter is completely independent of any UI framework; it only uses the abstract View interface.

#### Composition

```typescript
// main.ts
import { TodoModel } from './model/todo.model';
import { ConsoleTodoView } from './view/console-todo.view';
import { TodoPresenter } from './presenter/todo.presenter';

const model = new TodoModel();
const view = new ConsoleTodoView();
const presenter = new TodoPresenter(model, view); // view events are wired inside presenter
```

#### Testing the Presenter

```typescript
// test/todo.presenter.test.ts
import { TodoPresenter } from '../presenter/todo.presenter';
import { TodoModel } from '../model/todo.model';

describe('TodoPresenter', () => {
  it('should add a task and update view', () => {
    const model = new TodoModel();
    const view = {
      setTodos: jest.fn(),
      setErrorMessage: jest.fn(),
      clearInput: jest.fn(),
      onAddTask: () => {},
      onRemoveTask: () => {}
    };
    const presenter = new TodoPresenter(model, view);
    
    // Simulate user adding task
    view.onAddTask('Buy milk');
    
    expect(model.getTodos()).toEqual(['Buy milk']);
    expect(view.setTodos).toHaveBeenCalledWith(['Buy milk']);
    expect(view.clearInput).toHaveBeenCalled();
    expect(view.setErrorMessage).not.toHaveBeenCalled();
  });
});
```

**Explanation**:
- The test creates a mock View and a real Model.
- It triggers the presenter via the View's event callback and asserts that the Model was updated and the View was refreshed appropriately.
- The presenter is tested in isolation, without any UI.

### Benefits of MVP

- **Maximum Testability** – The Presenter contains all logic and can be unit‑tested with mock Views.
- **View Isolation** – The View is completely passive, making it easy to swap (e.g., from console to web) without changing the Presenter.
- **Clear Boundaries** – The View interface defines a clear contract.

### Drawbacks

- **Boilerplate** – You need to define an interface for every View, and the Presenter often has many simple forwarding methods.
- **Presenter Size** – The Presenter can become large if the View is complex.
- **Event Wiring** – Manually wiring events can be tedious, though frameworks can help.

MVP is often used in environments where testability is paramount, such as complex enterprise desktop applications (e.g., WinForms, Android with MVP pattern).

---

## 14.4 Comparing MVC, MVVM, and MVP

| Aspect | MVC | MVVM | MVP |
|--------|-----|------|-----|
| **View's Knowledge** | Knows Model (observes it) | Knows ViewModel (via binding) | Knows Presenter (via interface) |
| **Model's Knowledge** | None | None | None |
| **Controller/Presenter/ViewModel Role** | Handles input, updates Model | Exposes bindable data and commands | Contains all presentation logic, updates View |
| **Testability** | Medium (Controller can be tested, but View often needs UI) | High (ViewModel is UI‑independent) | Very High (Presenter fully testable) |
| **Data Binding** | Not built‑in (manual updates) | Central feature | Not used (manual updates via Presenter) |
| **Typical Platforms** | Web frameworks (Ruby on Rails, ASP.NET MVC), iOS | WPF, Xamarin, Vue, Angular | WinForms, Android, GWT |
| **Complexity** | Moderate | Moderate to High (requires binding framework) | Moderate (requires View interfaces) |

### Choosing a Pattern

- **Use MVC** when you have a framework that enforces it (like Rails or ASP.NET MVC) and you want a straightforward separation with some automated View updates.
- **Use MVVM** when your platform supports robust data binding (WPF, Vue) and you want to minimise manual UI update code.
- **Use MVP** when testability is critical and you can afford the boilerplate of defining View interfaces, especially in environments without built‑in data binding.

In modern web development, frameworks like React and Vue blur the lines: React's one‑way data flow resembles MVP with a "passive view" (the render function) and a "presenter" (the component logic), while Vue's two‑way binding is closer to MVVM. Understanding the underlying patterns helps you use these frameworks more effectively.

---

## Chapter Summary

This chapter explored three foundational patterns for separating UI concerns:

1. **Model‑View‑Controller (MVC)** – The classic pattern where the Model notifies the View of changes, and the Controller handles input. It provides a clean separation but can lead to Controller bloat and View‑Model coupling.

2. **Model‑View‑ViewModel (MVVM)** – An evolution that leverages data binding to automatically synchronise the View and ViewModel. The ViewModel exposes observable data and commands, making the View completely passive and the ViewModel highly testable.

3. **Model‑View‑Presenter (MVP)** – A pattern that maximises testability by making the View a passive interface and placing all presentation logic in the Presenter. The Presenter mediates between Model and View, updating the View explicitly.

All three patterns aim to **separate the user interface from business logic**, making applications more maintainable, testable, and adaptable. The choice among them depends on your platform, team preferences, and the importance of testability versus development speed.

**Key Insight**: Regardless of the pattern, the underlying principle is the same—**separate concerns**. The Model should never know about the View; the View should contain minimal logic; and the middle component (Controller, ViewModel, or Presenter) should handle the interaction.

---

## Next Chapter Preview

**Chapter 15: Service-Oriented Architecture (SOA) and Microservices**

We now move from UI‑level separation to system‑level architecture. Chapter 15 will explore **Service‑Oriented Architecture** principles, the decision between **monoliths and microservices**, the **Service Layer pattern**, and the **Backend for Frontend (BFF)** pattern. You'll learn how to decompose large systems into services that can be developed, deployed, and scaled independently.



<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='13. layered_and_modular_architecture.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='15. service_oriented_architecture.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
